<a href="https://colab.research.google.com/github/MariamAllam816/Momentum-AI/blob/colab_notebooks/Momentum_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run in Google Colab first cell
!pip install transformers torch torchvision
!pip install sentence-transformers
!pip install pillow requests
!pip install gradio  # for quick interface testing

import torch
from transformers import CLIPProcessor, CLIPModel
from transformers import pipeline
import json
import numpy as np
from PIL import Image
import requests

In [ ]:
# In Google Colab (use ! prefix)
!pip install requests pillow numpy transformers torch torchvision
!pip install sentence-transformers gradio
# Note: tkinter comes pre-installed in Colab, PyQt5 not needed for Colab

In [ ]:
import json
import random
from datetime import datetime, timedelta
import os

# Create directory structure in Colab
os.makedirs('data/mock_profiles', exist_ok=True)

# Define 10 place categories
CATEGORIES = {
    "restaurant": ["pizza place", "cafe", "fine dining", "food truck"],
    "outdoor": ["park", "beach", "hiking trail", "garden"],
    "entertainment": ["movie theater", "concert hall", "arcade", "bar"],
    "shopping": ["mall", "boutique", "market", "bookstore"],
    "fitness": ["gym", "yoga studio", "swimming pool", "sports court"],
    "cultural": ["museum", "gallery", "theater", "library"],
    "travel": ["airport", "hotel", "landmark", "tourist spot"],
    "work": ["office", "coworking space", "conference room", "business center"],
    "social": ["party venue", "club", "community center", "event space"],
    "wellness": ["spa", "salon", "meditation center", "health clinic"]
}

def generate_mock_profile():
    """Generate a mock social media profile with posts"""

    profile = {
        "username": "demo_user_2024",
        "bio": "Love exploring new places! 📍✈️",
        "posts": []
    }

    # Generate 20 mock posts
    for i in range(20):
        category = random.choice(list(CATEGORIES.keys()))
        place_type = random.choice(CATEGORIES[category])

        post = {
            "id": f"post_{i}",
            "image_url": f"https://example.com/image_{i}.jpg",
            "caption": f"Amazing time at this {place_type}! #goodvibes",
            "location": f"Sample {place_type.title()}",
            "date": (datetime.now() - timedelta(days=random.randint(1, 365))).isoformat(),
            "true_category": category
        }
        profile["posts"].append(post)

    return profile

# Generate and save mock data
mock_profile = generate_mock_profile()
with open('data/mock_profiles/demo_user.json', 'w') as f:
    json.dump(mock_profile, f, indent=2)

print("Mock profile created!")
print(f"Generated {len(mock_profile['posts'])} posts for user: {mock_profile['username']}")

# Let's also display a sample of what we created
print("\nSample posts:")
for i, post in enumerate(mock_profile['posts'][:3]):
    print(f"{i+1}. {post['caption']} - {post['location']} ({post['true_category']})")

Mock profile created!
Generated 20 posts for user: demo_user_2024

Sample posts:
1. Amazing time at this library! #goodvibes - Sample Library (cultural)
2. Amazing time at this fine dining! #goodvibes - Sample Fine Dining (restaurant)
3. Amazing time at this salon! #goodvibes - Sample Salon (wellness)


In [ ]:
# File: colab_notebooks/image_analysis.ipynb

# Cell 1: Setup
from transformers import CLIPProcessor, CLIPModel, pipeline
import torch
from PIL import Image
import json
import numpy as np

# Load CLIP model for image understanding
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Cell 2: Category Classification Function
def classify_place_from_image(image_path):
    """Classify an image into one of our 10 categories"""

    # Load image
    image = Image.open(image_path)

    # Define category labels for CLIP
    category_labels = [
        "restaurant and food place",
        "outdoor nature and parks",
        "entertainment and fun venue",
        "shopping and retail store",
        "fitness and gym facility",
        "cultural museum and gallery",
        "travel and transportation",
        "office and workplace",
        "social party and events",
        "wellness spa and health"
    ]

    # Process with CLIP
    inputs = processor(
        text=category_labels,
        images=image,
        return_tensors="pt",
        padding=True
    )

    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)

    # Get top prediction
    predicted_idx = probs.argmax().item()
    confidence = probs[0][predicted_idx].item()

    category_mapping = [
        "restaurant", "outdoor", "entertainment", "shopping", "fitness",
        "cultural", "travel", "work", "social", "wellness"
    ]

    return {
        "category": category_mapping[predicted_idx],
        "confidence": confidence,
        "all_scores": dict(zip(category_mapping, probs[0].tolist()))
    }

# Cell 3: Process Mock Profile
def analyze_profile(profile_path):
    """Analyze all images in a mock profile"""

    with open(profile_path, 'r') as f:
        profile = json.load(f)

    results = {
        "username": profile["username"],
        "analyzed_posts": [],
        "category_summary": {}
    }

    # In real demo, you'd download images or use sample images
    # For now, we'll simulate results
    for post in profile["posts"]:
        # Simulate image analysis (replace with actual image processing)
        analysis = {
            "post_id": post["id"],
            "predicted_category": post["true_category"],  # Using mock data
            "confidence": random.uniform(0.7, 0.95),
            "location": post["location"]
        }
        results["analyzed_posts"].append(analysis)

    # Create category summary
    categories = [p["predicted_category"] for p in results["analyzed_posts"]]
    for cat in set(categories):
        results["category_summary"][cat] = categories.count(cat)

    return results

# Test the analysis
analysis_results = analyze_profile('data/mock_profiles/demo_user.json')
print("Analysis complete!")
print(f"Categories found: {analysis_results['category_summary']}")
print("Analysis complete!")
print(f"Categories found: {analysis_results['category_summary']}")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Analysis complete!
Categories found: {'cultural': 1, 'social': 1, 'fitness': 2, 'outdoor': 4, 'travel': 3, 'shopping': 1, 'wellness': 4, 'restaurant': 4}
Analysis complete!
Categories found: {'cultural': 1, 'social': 1, 'fitness': 2, 'outdoor': 4, 'travel': 3, 'shopping': 1, 'wellness': 4, 'restaurant': 4}


In [ ]:
# Cell 4: Recommendation System
def generate_recommendations(analysis_results):
    """Generate place recommendations based on user's patterns"""

    # Sample places database (in real app, this would be much larger)
    places_db = {
        "restaurant": ["Tony's Italian Bistro", "Green Garden Cafe", "Sushi Master"],
        "outdoor": ["Central Park Trail", "Sunset Beach", "Mountain View Hike"],
        "entertainment": ["Retro Gaming Lounge", "Live Music Hall", "Comedy Club Downtown"],
        "shopping": ["Vintage Market", "Tech Store Plus", "Artisan Crafts Shop"],
        "fitness": ["PowerFit Gym", "Zen Yoga Studio", "AquaFit Center"],
        "cultural": ["Modern Art Museum", "History Gallery", "Cultural Center"],
        "travel": ["Airport Lounge VIP", "Boutique Hotel", "City Landmark Tour"],
        "work": ["Creative Coworking", "Business Hub", "Conference Center"],
        "social": ["Rooftop Lounge", "Community Events", "Social Club"],
        "wellness": ["Serenity Spa", "Wellness Retreat", "Mindful Center"]
    }

    recommendations = []

    # Get user's top 3 categories
    top_categories = sorted(
        analysis_results["category_summary"].items(),
        key=lambda x: x[1],
        reverse=True
    )[:3]

    for category, count in top_categories:
        places = places_db.get(category, [])
        for place in places[:2]:  # Top 2 places per category
            recommendations.append({
                "place_name": place,
                "category": category,
                "reason": f"Based on your {count} visits to {category} places",
                "match_score": random.uniform(0.8, 0.95)
            })

    return recommendations

# Generate recommendations
recommendations = generate_recommendations(analysis_results)
print("Recommendations generated!")
for rec in recommendations[:5]:
    print(f"- {rec['place_name']} ({rec['category']}) - {rec['reason']}")

Recommendations generated!
- Central Park Trail (outdoor) - Based on your 4 visits to outdoor places
- Sunset Beach (outdoor) - Based on your 4 visits to outdoor places
- Serenity Spa (wellness) - Based on your 4 visits to wellness places
- Wellness Retreat (wellness) - Based on your 4 visits to wellness places
- Tony's Italian Bistro (restaurant) - Based on your 4 visits to restaurant places
